In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nishchalverma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
twitter_data=pd.read_csv('/Users/nishchalverma/Downloads/twitter_sa.csv', sep=',', encoding='latin-1')
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [7]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

In [8]:
twitter_data.describe()

,0,1467810369
count,1.599999e+06,1.599999e+06
mean,2.000001e+00,1.998818e+09
std,2.000001e+00,1.935757e+08
min,0.000000e+00,1.467811e+09
25%,0.000000e+00,1.956916e+09
50%,4.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


In [9]:
twitter_data.shape

(1599999, 6)

In [18]:
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/Users/nishchalverma/Downloads/twitter_sa.csv', sep=',', names=column_names, encoding='latin-1')

In [11]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [19]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [20]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [21]:
twitter_data['target'].value_counts()
#0-->Negative tweets
#1-->Positive tweets

target
0    800000
1    800000
Name: count, dtype: int64

In [22]:
portS=PorterStemmer()

In [29]:
def Stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ', content)
    stemmed_content=stemmed_content.lower().split()
    stemmed_content=[portS.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [30]:
twitter_data['stemmed_content']=twitter_data['text'].apply(Stemming)

In [32]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [34]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [35]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [36]:
print(Y)

[0 0 0 ... 1 1 1]


In [38]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.3,stratify=Y, random_state=42)

In [39]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1120000,) (480000,)


In [40]:
# Textual to numerical conversion
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [41]:
print(X_train)

  (0, 37734)	0.2370816726175219
  (0, 153109)	0.26109664702445023
  (0, 114162)	0.23079857294575395
  (0, 61034)	0.2530840904948045
  (0, 21527)	0.3635478715103337
  (0, 198336)	0.28665036448368497
  (0, 359280)	0.3450902555028528
  (0, 136062)	0.2364424015472906
  (0, 291637)	0.22264901619038097
  (0, 112037)	0.5651556589513604
  (1, 145600)	0.46571119564116675
  (1, 26374)	0.3029474218759125
  (1, 295816)	0.3236508208308959
  (1, 8791)	0.7658890833591377
  (2, 148748)	0.330769196609809
  (2, 13646)	0.3722922363896325
  (2, 233325)	0.35803608369152534
  (2, 219324)	0.4055713900329562
  (2, 43171)	0.39909724657997897
  (2, 215037)	0.16897603366597974
  (2, 307524)	0.2943887324780582
  (2, 262411)	0.3480512551250672
  (2, 136062)	0.25234168398086554
  (3, 377337)	0.47638149741992336
  (3, 399678)	0.37908928001474057
  :	:
  (1119997, 411189)	0.49189680120205265
  (1119997, 375746)	0.4489406569960318
  (1119997, 288839)	0.43245889643998114
  (1119997, 261453)	0.30734552096370527
  (11199

In [42]:
LR=LogisticRegression(max_iter=1000)

<IPython.core.display.Javascript object>

In [43]:
LR.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [45]:
pred=LR.predict(X_train)

In [46]:
accuracy_score(Y_train,pred)

0.8106669642857143

In [47]:
X_test_pred=LR.predict(X_test)

In [49]:
accuracy_score(Y_test,X_test_pred)

0.7775833333333333

In [ ]:
#test Manually

In [50]:
x=X_train[200]
y=Y_test[200]
print(y)

1


In [52]:
prediction=LR.predict(x)
print(prediction)

[1]
